### Functools:

In [8]:
from functools import reduce, cmp_to_key, cache, _make_key, lru_cache, total_ordering, wraps
from functools import partial, partialmethod, singledispatch, singledispatchmethod, update_wrapper

def print_details(*args):
    print(args[0])
    print(args[1])
    print("-"*25)

data = [1,2,3,4]
data2 = [5,6,7,8]
string_data = "abcdefg"

# reduce
print_details("Reduce:", reduce(lambda x,y: x*y, data))

# cmp_to_key
def mycmp(a, b): 
    print("comparing ", a, " and ", b) 
    if a > b: 
        return 1
    elif a < b: 
        return -1
    else: 
        return 0

data = [1,2,4,3,8,5]
print_details("cmp_to_key:", sorted(data, key=cmp_to_key(mycmp)))

# partial function
def add(a, b, c):
    return 100 * a + 10 * b + c

add_part = partial(add, c = 2, b = 1)
print_details("partial function: ", add_part(3))

Reduce:
24
-------------------------
comparing  2  and  1
comparing  4  and  2
comparing  3  and  4
comparing  3  and  2
comparing  3  and  4
comparing  8  and  3
comparing  8  and  4
comparing  5  and  3
comparing  5  and  8
comparing  5  and  4
cmp_to_key:
[1, 2, 3, 4, 5, 8]
-------------------------
partial function: 
312
-------------------------


In [16]:
# partial method => mostly applied inside class
# it is mandatory to initialize the value of args while using partial method
class Switch:
    def __init__(self):
        self._state = False

    def set_new_state(self, new_state: bool):
        self._state = new_state

    @property
    def state(self):
        return self._state

    turn_on = partialmethod(set_new_state, True)
    turn_off = partialmethod(set_new_state, False)

switch = Switch()
print("Turning on the switch: ", end="")
switch.turn_on()
print(switch.state)

print("Turning off the switch: ", end="")
switch.turn_off()
print(switch.state)

Turning on the switch: True
Turning off the switch: False


In [15]:
# total ordering => used as class decorators when we want to avoid implementing the comparison methods
@total_ordering
class Students: 
	def __init__(self, cgpa): 
		self.cgpa = cgpa 

	def __lt__(self, other): 
		return self.cgpa<other.cgpa 

	def __eq__(self, other): 
		return self.cgpa == other.cgpa 

Arjun = Students(8.6) 
Ram = Students(7.5) 
print(Arjun.__lt__(Ram)) 
print(Arjun.__le__(Ram)) 
print(Arjun.__gt__(Ram)) 
print(Arjun.__ge__(Ram)) 
print(Arjun.__eq__(Ram)) 
print(Arjun.__ne__(Ram)) 


False
False
True
True
False
True


In [25]:
# update wrapper

def hi(func):
	def wrapper():
		"Hi has taken over Hello Documentation"
		print("Hi geeks")
		func()
	return wrapper
	
@hi
def hello():
	"this is the documentation of Hello Function"
	print("Hey Geeks")

print(hello.__name__)
print(hello.__doc__)
help(hello)


wrapper
Hi has taken over Hello Documentation
Help on function wrapper in module __main__:

wrapper()
    Hi has taken over Hello Documentation



In [39]:
# alternate for update wrapper with cons

def hi(func):
	def wrapper():
		"Hi has taken over Hello Documentation"
		print("Hi geeks")
		func()
  
	wrapper.__doc__ = func.__doc__
	wrapper.__name__ = func.__name__
	return wrapper
	
@hi
def hello():
	"this is the documentation of Hello Function"
	print("Hey Geeks")

print(hello.__name__)
print(hello.__doc__)
help(hello)


hello
this is the documentation of Hello Function
Help on function hello in module __main__:

hello()
    this is the documentation of Hello Function



In [37]:
# update wrapper => used to update the metadata of the wrapper function, allows for better readability and re-usability of code
import functools

def hi(func):
	def wrapper(*args, **kwargs):
		"Hi has taken over Hello Documentation"
		print("Hi geeks")
		func()
		
	print("UPDATED WRAPPER DATA")
	print(f'WRAPPER ASSIGNMENTS : {functools.WRAPPER_ASSIGNMENTS}')
	print(f'UPDATES : {functools.WRAPPER_UPDATES}')
	
	update_wrapper(wrapper, func)
	return wrapper
	
@hi
def hello():
	"this is the documentation of Hello Function"
	print("Hey Geeks")

print(hello.__name__)
print(hello.__doc__)
help(hello)


UPDATED WRAPPER DATA
WRAPPER ASSIGNMENTS : ('__module__', '__name__', '__qualname__', '__doc__', '__annotations__')
UPDATES : ('__dict__',)
wrapper
Hi has taken over Hello Documentation
Help on function wrapper in module __main__:

wrapper(*args, **kwargs)
    Hi has taken over Hello Documentation



In [22]:
# update wrapper

def divide(a, b):
	"Original Documentation of Divide"
	return a / b

half = functools.partial(divide, b = 2)
oneThird = functools.partial(divide, b = 3)

print("UPDATED WRAPPER DATA")
print(f'WRAPPER ASSIGNMENTS : {functools.WRAPPER_ASSIGNMENTS}')
print(f'UPDATES : {functools.WRAPPER_UPDATES}')
update_wrapper(half, divide)
try:
	print(half.__name__)
	
except AttributeError:
	print('No Name')
	
print(half.__doc__)
help(half)
help(oneThird)


UPDATED WRAPPER DATA
WRAPPER ASSIGNMENTS : ('__module__', '__name__', '__qualname__', '__doc__', '__annotations__')
UPDATES : ('__dict__',)
divide
Original Documentation of Divide
Help on partial in module __main__:

divide = functools.partial(<function divide at 0x10c1a3c40>, b=2)
    Original Documentation of Divide

Help on partial in module functools object:

class partial(builtins.object)
 |  partial(func, *args, **keywords) - new function with partial application
 |  of the given arguments and keywords.
 |  
 |  Methods defined here:
 |  
 |  __call__(self, /, *args, **kwargs)
 |      Call self as a function.
 |  
 |  __delattr__(self, name, /)
 |      Implement delattr(self, name).
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __reduce__(...)
 |      Helper for pickle.
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  __setattr__(self, name, value, /)
 |      Implement setattr(self, name, value).
 |  
 |  __setstate__(...)
 | 

In [35]:
# wraps => similar to update_wrapper but better since it can work for multiple functions (decorator)
# i.e update_wrapper(wrapper,divide) => can be applied to only one func called divide and if new func used then, we duplicate

def a_decorator(func):
	@wraps(func)
	def wrapper(*args, **kwargs):
		"""A wrapper function"""
		func()
	return wrapper

@a_decorator
def first_function():
	"""This is docstring for first function"""
	print("first function")

@a_decorator
def second_function(a):
	"""This is docstring for second function"""
	print("second function")

print(first_function.__name__)
print(first_function.__doc__)
help(first_function)
print(second_function.__name__)
print(second_function.__doc__)
help(second_function)

first_function
This is docstring for first function
Help on function first_function in module __main__:

first_function()
    This is docstring for first function

second_function
This is docstring for second function
Help on function second_function in module __main__:

second_function(a)
    This is docstring for second function



In [31]:
import time 

# without lru_cache
def fib_without_cache(n): 
	if n < 2: 
		return n 
	return fib_without_cache(n-1) + fib_without_cache(n-2) 
	
begin = time.time() 
fib_without_cache(30) 
end = time.time() 

print("Time taken to execute the function without lru_cache is", end-begin) 

# with lru_cache
@lru_cache(maxsize = 128, typed=True) 
def fib_with_cache(n): 
	if n < 2: 
		return n 
	return fib_with_cache(n-1) + fib_with_cache(n-2) 
	
begin = time.time() 
fib_with_cache(30) 
end = time.time() 

print("Time taken to execute the function with lru_cache is", end-begin) 


Time taken to execute the function without lru_cache is 0.677175760269165
Time taken to execute the function with lru_cache is 0.0001499652862548828


In [34]:
# cache => if we dont want to specify the maxsize then use cache or else use lru_cache
@cache 
def fib_with_cache(n): 
	if n < 2: 
		return n 
	return fib_with_cache(n-1) + fib_with_cache(n-2) 
	
begin = time.time() 

fib_with_cache(30) 
end = time.time() 

print("Time taken to execute the function with lru_cache is", end-begin) 

Time taken to execute the function with lru_cache is 6.985664367675781e-05


In [50]:
# singledispatch => function overloading
from decimal import Decimal

@singledispatch
def fun(s): 
	print(s) 

@fun.register(int) 
def _1(s): 
	print(s * 2) 

@fun.register(Decimal) 
def _3(s): 
	print(s * 3) 

@fun.register(list) 
def _2(s): 
	for i, e in enumerate(s):print(i, e) 

@fun.register(float) 
def _4(s): 
	print(round(s*2))

fun('GeeksforGeeks') 
fun(10) 
fun(['g', 'e', 'e', 'k', 's']) 
fun(Decimal(20.01))
fun(10.01)

print(fun.dispatch(dict)) 
print(fun.dispatch(list))
print(fun.registry.keys()) 
print(fun.registry[int]) 
print(fun.registry[object])


GeeksforGeeks
20
0 g
1 e
2 e
3 k
4 s
60.03000000000000468958205602
20
<function fun at 0x10c2cd4e0>
<function _2 at 0x10c2e7880>
dict_keys([<class 'object'>, <class 'int'>, <class 'decimal.Decimal'>, <class 'list'>, <class 'float'>])
<function _1 at 0x10c2e6700>
<function fun at 0x10c2cd4e0>


In [ ]:
class Sum:
    @singledispatchmethod
    def sumMethod(self, arg1, arg2):
        print("Default implementation with arg1 = %s and arg2 = %s" % (arg1, arg2))

    @sumMethod.register
    def _(self, arg1: int, arg2: int):
        print("Sum with arg1 as integer. %s + %s = %s" % (arg1, arg2, arg1 + arg2))

    @sumMethod.register
    def _(self, arg1: float, arg2: float):
        print("Sum with arg1 as float. %s + %s = %s" % (arg1, arg2, arg1 + arg2))


s = Sum()
s.sumMethod(2, 3)
s.sumMethod(2.1, 3.4)
s.sumMethod("hi", 3.4)